## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

import gmaps.datasets
import numpy as np

# # Import API key
# from config import g_key

# Import API key
import os
from dotenv import load_dotenv
load_dotenv()
g_key = os.getenv("GMaps_Key")

# # Configure gmaps
# gmaps.configure(api_key=g_key)

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,4,Georgetown,MY,80.89,thunderstorm,5.4112,100.3354,Cititel Penang
1,5,Norman Wells,CA,73.44,few clouds,65.2820,-126.8329,Heritage Hotel
2,6,Tomatlan,MX,81.66,overcast clouds,19.9333,-105.2500,El Salvial
3,14,Sao Filipe,CV,75.49,few clouds,14.8961,-24.4956,Hotel Xaguate
4,20,Atuona,PF,77.05,few clouds,-9.8000,-139.0333,Villa Enata


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]


In [4]:
locations.head(10)

,Lat,Lng
0,5.4112,100.3354
1,65.2820,-126.8329
2,19.9333,-105.2500
3,14.8961,-24.4956
4,-9.8000,-139.0333
5,25.1833,73.4333
6,13.9228,43.9176
7,34.2257,-77.9447
8,39.3558,-9.3811
9,39.0500,-27.9833


In [5]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
                                 dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] == "Narbonne"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Narbonne"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Fontaine"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Lerici"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Codigoro"]


In [7]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]


In [8]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
vacation_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='DRIVING')

fig.add_layer(vacation_itinerary)

fig


Figure(layout=FigureLayout(height='420px'))

In [9]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, 
                          vacation_stop3, vacation_end], ignore_index=True)
itinerary_df


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,125,Narbonne,FR,82.63,clear sky,43.0833,2.8333,Domaine du Soleil Couchant - Bed & Breakfast in
1,289,Fontaine,FR,75.47,clear sky,45.1984,5.6827,Hôtel Institut
2,475,Lerici,IT,81.36,clear sky,44.0759,9.9112,Doria Park Hotel
3,270,Codigoro,IT,75.09,clear sky,44.8311,12.1142,casa maria
4,125,Narbonne,FR,82.63,clear sky,43.0833,2.8333,Domaine du Soleil Couchant - Bed & Breakfast in


In [10]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} </dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]


In [11]:
# 11a. Add a marker layer for each city to the map.

locations = itinerary_df[["Lat", "Lng"]]

fig = gmaps.figure()

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))